In [1]:
import pandas as pd
import numpy as np 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from datetime import datetime
from sklearn import preprocessing, model_selection, metrics, ensemble
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [2]:
train=pd.read_csv(r'C:\Users\My PC\Desktop\Machine Learning\Hackerearth\dataset\health\train.csv')
test=pd.read_csv(r'C:\Users\My PC\Desktop\Machine Learning\Hackerearth\dataset\health\test.csv')
sub=pd.read_csv(r'C:\Users\My PC\Desktop\Machine Learning\Hackerearth\dataset\health\sample.csv')
# dd=pd.read_csv(r'C:\Users\My PC\Desktop\Machine Learning\Hackerearth\dataset\health\Data_Dictionary.xlsx')
first=pd.read_csv(r'C:\Users\My PC\Desktop\Machine Learning\Hackerearth\dataset\health\First_Health_Camp_Attended.csv')
second=pd.read_csv(r'C:\Users\My PC\Desktop\Machine Learning\Hackerearth\dataset\health\Second_Health_Camp_Attended.csv')
third=pd.read_csv(r'C:\Users\My PC\Desktop\Machine Learning\Hackerearth\dataset\health\Third_Health_Camp_Attended.csv')
hdet=pd.read_csv(r'C:\Users\My PC\Desktop\Machine Learning\Hackerearth\dataset\health\Health_Camp_Detail.csv')
pp=pd.read_csv(r'C:\Users\My PC\Desktop\Machine Learning\Hackerearth\dataset\health\Patient_Profile.csv')

In [3]:
train.head(2)

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,489652,6578,10-Sep-05,4,0,0,0,2
1,507246,6578,18-Aug-05,45,5,0,0,7


In [4]:
first.head(2)

,Patient_ID,Health_Camp_ID,Donation,Health_Score,Unnamed: 4
0,506181,6560,40,0.439024,NaN
1,494977,6560,20,0.097561,NaN


In [5]:
second.head(2)

,Patient_ID,Health_Camp_ID,Health Score
0,526631,6536,0.875136
1,509122,6536,0.755700


In [6]:
third.head(2)

,Patient_ID,Health_Camp_ID,Number_of_stall_visited,Last_Stall_Visited_Number
0,517875,6527,3,1
1,504692,6578,1,1


In [7]:
hdet.head(2)

,Health_Camp_ID,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3
0,6560,16-Aug-03,20-Aug-03,First,B,2
1,6530,16-Aug-03,28-Oct-03,First,C,2


In [8]:
pp.head(2)

,Patient_ID,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category
0,516956,0,0,0,0,1,90,39,18-Jun-03,NaN,Software Industry
1,507733,0,0,0,0,1,None,40,20-Jul-03,H,Software Industry


Train Data Analysis

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75278 entries, 0 to 75277
Data columns (total 8 columns):
Patient_ID           75278 non-null int64
Health_Camp_ID       75278 non-null int64
Registration_Date    74944 non-null object
Var1                 75278 non-null int64
Var2                 75278 non-null int64
Var3                 75278 non-null int64
Var4                 75278 non-null int64
Var5                 75278 non-null int64
dtypes: int64(7), object(1)
memory usage: 4.6+ MB


In [10]:
train.isna().sum()

Patient_ID             0
Health_Camp_ID         0
Registration_Date    334
Var1                   0
Var2                   0
Var3                   0
Var4                   0
Var5                   0
dtype: int64

In [11]:
train.Patient_ID.nunique()

29828

In [12]:
train.groupby('Patient_ID')['Health_Camp_ID'].count().unique()

array([ 2,  1,  5,  4,  8, 14,  3,  6,  7,  9, 12, 13, 10, 19, 11, 22, 16,
       18, 20, 15, 23, 17, 28, 24, 21, 32, 25], dtype=int64)

In [13]:
train.Registration_Date.isnull().sum()

334

In [14]:
train.Registration_Date.nunique()

1201

In [15]:
train.Registration_Date = pd.to_datetime(train.Registration_Date)
train.Registration_Date=train.Registration_Date.fillna(method='bfill')

Test Data

In [16]:
test.info(),test.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35249 entries, 0 to 35248
Data columns (total 8 columns):
Patient_ID           35249 non-null int64
Health_Camp_ID       35249 non-null int64
Registration_Date    35249 non-null object
Var1                 35249 non-null int64
Var2                 35249 non-null int64
Var3                 35249 non-null int64
Var4                 35249 non-null int64
Var5                 35249 non-null int64
dtypes: int64(7), object(1)
memory usage: 2.2+ MB


(None, Patient_ID           0
 Health_Camp_ID       0
 Registration_Date    0
 Var1                 0
 Var2                 0
 Var3                 0
 Var4                 0
 Var5                 0
 dtype: int64)

In [17]:
test.Patient_ID.nunique()

15324

In [18]:
test.groupby('Patient_ID')['Health_Camp_ID'].count().unique()

array([ 1,  5,  2,  4,  8,  3,  7,  6,  9, 10, 12, 13, 15, 14, 11, 17],
      dtype=int64)

In [19]:
test.Registration_Date = pd.to_datetime(test.Registration_Date)

Meta data

In [20]:
first.isnull().sum()

Patient_ID           0
Health_Camp_ID       0
Donation             0
Health_Score         0
Unnamed: 4        6218
dtype: int64

In [21]:
first.drop('Unnamed: 4',axis=1,inplace = True)

In [22]:
second.isnull().sum()

Patient_ID        0
Health_Camp_ID    0
Health Score      0
dtype: int64

In [23]:
third.isnull().sum()

Patient_ID                   0
Health_Camp_ID               0
Number_of_stall_visited      0
Last_Stall_Visited_Number    0
dtype: int64

pp

In [24]:
pp.isnull().sum()

Patient_ID               0
Online_Follower          0
LinkedIn_Shared          0
Twitter_Shared           0
Facebook_Shared          0
Income                   0
Education_Score          0
Age                      0
First_Interaction        0
City_Type            23384
Employer_Category    34793
dtype: int64

In [25]:
pp.drop(['City_Type','Employer_Category'],axis=1,inplace = True)

In [26]:
hdet.isnull().sum()

Health_Camp_ID     0
Camp_Start_Date    0
Camp_End_Date      0
Category1          0
Category2          0
Category3          0
dtype: int64

In [27]:
pp.replace('None',-999,inplace=True)

In [28]:
pp['Income'] = pp['Income'].astype('int64')
pp['Age'] = pp['Age'].astype('int64')
pp['Education_Score'] = pp['Education_Score'].astype('float64')
pp.First_Interaction = pd.to_datetime(pp.First_Interaction)

hdet

In [29]:
hdet.Camp_Start_Date= pd.to_datetime(hdet.Camp_Start_Date)
hdet.Camp_End_Date= pd.to_datetime(hdet.Camp_End_Date)

In [30]:
hdet['duration'] = hdet['Camp_End_Date'] - hdet['Camp_Start_Date']

In [31]:
second.rename(columns = {'Health_Score':'Health_Score2'},inplace = True)

Target Variable

In [32]:
first['Outcome_1'] = 1
second['Outcome_2'] = 1
third['Outcome_3'] = np.where(third['Number_of_stall_visited']>0,1,0)

In [33]:
train = pd.merge(train,pp,on='Patient_ID',how = 'left')
train = pd.merge(train,hdet,on='Health_Camp_ID',how = 'left')
train = pd.merge(train,first,on=['Patient_ID','Health_Camp_ID'],how = 'left')
train = pd.merge(train,second,on=['Patient_ID','Health_Camp_ID'],how = 'left')
train = pd.merge(train,third,on=['Patient_ID','Health_Camp_ID'],how = 'left')
train.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Online_Follower,LinkedIn_Shared,...,Category3,duration,Donation,Health_Score,Outcome_1,Health Score,Outcome_2,Number_of_stall_visited,Last_Stall_Visited_Number,Outcome_3
0,489652,6578,2005-09-10,4,0,0,0,2,0,0,...,2,59 days,NaN,NaN,NaN,NaN,NaN,2.0,1.0,1.0
1,507246,6578,2005-08-18,45,5,0,0,7,0,0,...,2,59 days,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,523729,6534,2006-04-29,0,0,0,0,0,0,0,...,2,751 days,NaN,NaN,NaN,0.402054,1.0,NaN,NaN,NaN
3,524931,6535,2004-02-07,0,0,0,0,0,0,0,...,2,17 days,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,521364,6529,2006-02-28,15,1,0,0,7,0,0,...,2,4 days,NaN,NaN,NaN,0.845597,1.0,NaN,NaN,NaN


In [34]:
train['target'] = np.where((train['Outcome_1'] == 1) | (train['Outcome_2'] == 1) | (train['Outcome_3'] == 1),1,0)

In [35]:
train.isnull().sum()

Patient_ID                       0
Health_Camp_ID                   0
Registration_Date                0
Var1                             0
Var2                             0
Var3                             0
Var4                             0
Var5                             0
Online_Follower                  0
LinkedIn_Shared                  0
Twitter_Shared                   0
Facebook_Shared                  0
Income                           0
Education_Score                  0
Age                              0
First_Interaction                0
Camp_Start_Date                  0
Camp_End_Date                    0
Category1                        0
Category2                        0
Category3                        0
duration                         0
Donation                     69060
Health_Score                 69060
Outcome_1                    69060
Health Score                 67459
Outcome_2                    67459
Number_of_stall_visited      68763
Last_Stall_Visited_N

In [36]:
train.drop(['Health Score','Health_Score','Outcome_1','Outcome_2','Number_of_stall_visited','Last_Stall_Visited_Number','Outcome_3'],axis=1,inplace=True)

In [37]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75278 entries, 0 to 75277
Data columns (total 24 columns):
Patient_ID           75278 non-null int64
Health_Camp_ID       75278 non-null int64
Registration_Date    75278 non-null datetime64[ns]
Var1                 75278 non-null int64
Var2                 75278 non-null int64
Var3                 75278 non-null int64
Var4                 75278 non-null int64
Var5                 75278 non-null int64
Online_Follower      75278 non-null int64
LinkedIn_Shared      75278 non-null int64
Twitter_Shared       75278 non-null int64
Facebook_Shared      75278 non-null int64
Income               75278 non-null int64
Education_Score      75278 non-null float64
Age                  75278 non-null int64
First_Interaction    75278 non-null datetime64[ns]
Camp_Start_Date      75278 non-null datetime64[ns]
Camp_End_Date        75278 non-null datetime64[ns]
Category1            75278 non-null object
Category2            75278 non-null object
Category3  

In [38]:
train.drop('Donation',axis=1,inplace=True)

In [39]:
train.Camp_Start_Date= pd.to_datetime(train.Camp_Start_Date)
train.Camp_End_Date= pd.to_datetime(train.Camp_End_Date)
train.First_Interaction= pd.to_datetime(train.First_Interaction)

In [40]:
train['Camps_Registered_per_patient'] = train.groupby('Patient_ID')['Health_Camp_ID'].transform('count')
train['Patients_Registered_per_camp'] = train.groupby('Health_Camp_ID')['Patient_ID'].transform('count')
# train['CampStart_Registration_Duration'] = train['Camp_Start_Date'] - train['Registration_Date']
# train['CampEnd_Registration_Duration'] = train['Camp_End_Date'] - train['Registration_Date']
# train['Registration_FirstInteraction_Duration'] = train['Registration_Date'] - train['First_Interaction']
# train['CampStart_FirstInteraction_Duration'] = train['Camp_Start_Date'] - train['First_Interaction']

In [41]:
test = pd.merge(test,pp,on='Patient_ID',how = 'left')
test = pd.merge(test,hdet,on='Health_Camp_ID',how = 'left')
test.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Online_Follower,LinkedIn_Shared,...,Income,Education_Score,Age,First_Interaction,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,duration
0,505701,6548,2006-05-21,1,0,0,0,2,0,0,...,0,-999.0,44,2003-02-05,2006-06-13,2006-08-18,Third,G,2,66 days
1,500633,6584,2006-06-02,0,0,0,0,0,0,1,...,1,67.0,41,2004-12-11,2006-08-04,2006-08-09,Second,A,2,5 days
2,506945,6582,2006-08-10,0,0,0,0,0,0,0,...,-999,-999.0,-999,2006-04-19,2006-04-06,2007-11-07,First,F,2,580 days
3,497447,6551,2006-08-27,0,0,0,0,0,0,0,...,0,-999.0,47,2006-08-25,2006-11-13,2006-11-18,Second,D,2,5 days
4,496446,6533,2006-09-19,0,0,0,0,0,0,0,...,-999,-999.0,-999,2006-09-19,2006-09-20,2006-09-23,First,E,2,3 days


In [42]:
test['Camps_Registered_per_patient'] = test.groupby('Patient_ID')['Health_Camp_ID'].transform('count')
test['Patients_Registered_per_camp'] = test.groupby('Health_Camp_ID')['Patient_ID'].transform('count')
# test["CampStart_Registration_Duration"] = test["Camp_Start_Date"] - test["Registration_Date"]
# test["CampEnd_Registration_Duration"] = test["Camp_End_Date"] - test["Registration_Date"]
# test["Registration_FirstInteraction_Duration"] = test["Registration_Date"] - test["First_Interaction"]
# test["CampStart_FirstInteraction_Duration"] = test["Camp_Start_Date"] - test["First_Interaction"]


In [66]:
test.isnull().sum(),test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35249 entries, 0 to 35248
Data columns (total 24 columns):
Patient_ID                      35249 non-null int64
Health_Camp_ID                  35249 non-null int64
Registration_Date               35249 non-null datetime64[ns]
Var1                            35249 non-null int64
Var2                            35249 non-null int64
Var3                            35249 non-null int64
Var4                            35249 non-null int64
Var5                            35249 non-null int64
Online_Follower                 35249 non-null int64
LinkedIn_Shared                 35249 non-null int64
Twitter_Shared                  35249 non-null int64
Facebook_Shared                 35249 non-null int64
Income                          35249 non-null int64
Education_Score                 35249 non-null float64
Age                             35249 non-null int64
First_Interaction               35249 non-null datetime64[ns]
Camp_Start_Date        

(Patient_ID                      0
 Health_Camp_ID                  0
 Registration_Date               0
 Var1                            0
 Var2                            0
 Var3                            0
 Var4                            0
 Var5                            0
 Online_Follower                 0
 LinkedIn_Shared                 0
 Twitter_Shared                  0
 Facebook_Shared                 0
 Income                          0
 Education_Score                 0
 Age                             0
 First_Interaction               0
 Camp_Start_Date                 0
 Camp_End_Date                   0
 Category1                       0
 Category2                       0
 Category3                       0
 duration                        0
 Camps_Registered_per_patient    0
 Patients_Registered_per_camp    0
 dtype: int64, None)

Model

In [45]:
from sklearn.preprocessing import LabelEncoder
cat_columns = []
for col in train.columns:
    if train[col].dtype == 'object':
        print(col)
        cat_columns.append(col)
        enc = preprocessing.LabelEncoder()
        full_list = list(train[col].values) + list(test[col].values)
        enc.fit(full_list)
        train[col] = enc.transform(list(train[col].values))
        test[col]  = enc.transform(list(test[col].values))

Category1
Category2


In [47]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75278 entries, 0 to 75277
Data columns (total 25 columns):
Patient_ID                      75278 non-null int64
Health_Camp_ID                  75278 non-null int64
Registration_Date               75278 non-null datetime64[ns]
Var1                            75278 non-null int64
Var2                            75278 non-null int64
Var3                            75278 non-null int64
Var4                            75278 non-null int64
Var5                            75278 non-null int64
Online_Follower                 75278 non-null int64
LinkedIn_Shared                 75278 non-null int64
Twitter_Shared                  75278 non-null int64
Facebook_Shared                 75278 non-null int64
Income                          75278 non-null int64
Education_Score                 75278 non-null float64
Age                             75278 non-null int64
First_Interaction               75278 non-null datetime64[ns]
Camp_Start_Date        

In [67]:
train["duration"] = pd.to_numeric(train["duration"])
test["duration"] = pd.to_numeric(test["duration"])

In [56]:
cols_to_use = ['Income','Age','Category1','Category2','duration','Patients_Registered_per_camp','Camps_Registered_per_patient']
X = train[cols_to_use]
Y = train['target']

In [68]:
finalt=test[cols_to_use]

In [57]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X, Y, test_size=0.15, random_state=42)

In [59]:
import lightgbm as lgb
from math import sqrt 
from sklearn.metrics import mean_absolute_error
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_cv, label=y_cv)

param = {'objective': 'regression',
         'num_leaves':20,
         'boosting': 'gbdt',  
         'metric': 'mae',
         'learning_rate': 0.1, 
         'num_iterations': 1000,
         'num_leaves': 90,
         'max_depth': 6,
         'min_data_in_leaf': 11,
         'bagging_fraction': 0.90,
         'bagging_freq': 1,
         'bagging_seed': 142,
         'feature_fraction': 0.90,
         'feature_fraction_seed': 2,
         'early_stopping_round': 200,
         'max_bin': 500
         }

lgbm = lgb.train(params=param, verbose_eval=100, train_set=train_data, valid_sets=[test_data])

y_pred_lgbm = lgbm.predict(X_cv)
print('RMSLE:', sqrt(mean_absolute_error(np.expm1(y_cv), np.expm1(y_pred_lgbm))))

Training until validation scores don't improve for 200 rounds
[100]	valid_0's l1: 0.272365
[200]	valid_0's l1: 0.269823
[300]	valid_0's l1: 0.268632
[400]	valid_0's l1: 0.267865
[500]	valid_0's l1: 0.267315
[600]	valid_0's l1: 0.266771
[700]	valid_0's l1: 0.266567
[800]	valid_0's l1: 0.266323
[900]	valid_0's l1: 0.26627
[1000]	valid_0's l1: 0.266116
Did not meet early stopping. Best iteration is:
[986]	valid_0's l1: 0.266022
RMSLE: 0.6632159085785474


In [69]:
pred=lgbm.predict(finalt)

In [73]:

sub['Outcome']=abs(pred)

In [74]:
sub.head()

,Patient_ID,Health_Camp_ID,Outcome
0,505701,6548,0.485567
1,500633,6584,0.550148
2,506945,6582,0.239629
3,497447,6551,0.684222
4,496446,6533,0.158084


In [75]:
sub.to_csv(r'C:\Users\My PC\Desktop\Machine Learning\Hackerearth\dataset\health\hav.csv',index=False)

In [77]:

# k=['Registration_Date','Camp_Start_Date','Camp_End_Date','First_Interaction']
# for i in k:
#   train[i] = pd.to_datetime(train[i])
#   test[i] = pd.to_datetime(test[i])
# for i in k:
#   test[i+'_'+'year'] = test[i].dt.year
#   test[i+'_'+'day'] = test[i].dt.day
#   test[i+'_'+'weekofyear'] = test[i].dt.weekofyear
#   test[i+'_'+'month'] = test[i].dt.month
#   test[i+'_'+'dayofweek'] =test[i].dt.dayofweek
#   test[i+'_'+'weekend'] = (test[i].dt.weekday >=5).astype(int)
#   train[i+'_'+'year'] = train[i].dt.year
#   train[i+'_'+'day'] = train[i].dt.day
#   train[i+'_'+'weekofyear'] = train[i].dt.weekofyear
#   train[i+'_'+'month'] = train[i].dt.month
#   train[i+'_'+'dayofweek'] = train[i].dt.dayofweek
#   train[i+'_'+'weekend'] = (train[i].dt.weekday >=5).astype(int)
# for i in k:
#   del train[i]
#   del test[i]
# lgbm = LGBMRegressor(boosting_type='gbdt', num_leaves=800, max_depth=31, learning_rate=0.1, n_estimators=1000, min_child_samples=20, subsample=0.9)